<a href="https://colab.research.google.com/github/ykim68ncstate/ST-554-Project1/blob/main/Task1/Project1_Task1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task 1: Writing two gradient descent type algorithms
**Objectives**
* find the optimal constant to use for squared error loss.
* find the optimal intercept and slope from a simple linear regression model.

In [151]:
!pip install ucimlrepo  #install UCI data repo library
import ucimlrepo as uci
air_quality = uci.fetch_ucirepo(id=360)

## Dataset
This data is time series data (data recorded over time) for air quality measurements in Italy.

**Variable types**
* True/Gold standard measurements of pollutants in the air
  - CO(GT): CO concentration
  - NMHC(GT): Non Metanic HydroCarbon concentration
  - C6H6(GT): Benzene concentration
  - NOx(GT): NOx concentration
  - NO2(GT): NO2 concentration

* Sensor measurements of pollutants in the air
  - PT08.S1(CO)
  - PT08.S2(NMHC)
  - PT08.S3(NOx)
  - PT08.S4(NO2)
  - PT08.S5(O3)

* Weather related measurements
  - T: Temperature
  - RH: Relative humidity
  - AH: Absolute humidity

In [152]:
df = air_quality.data.features # set dataset as "df"

In [153]:
df.head() # Extract the first 5 rows of the dataset(to check the data columns)

,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
0,3/10/2004,18:00:00,2.6,1360,150,11.9,1046,166,1056,113,1692,1268,13.6,48.9,0.7578
1,3/10/2004,19:00:00,2.0,1292,112,9.4,955,103,1174,92,1559,972,13.3,47.7,0.7255
2,3/10/2004,20:00:00,2.2,1402,88,9.0,939,131,1140,114,1555,1074,11.9,54.0,0.7502
3,3/10/2004,21:00:00,2.2,1376,80,9.2,948,172,1092,122,1584,1203,11.0,60.0,0.7867
4,3/10/2004,22:00:00,1.6,1272,51,6.5,836,131,1205,116,1490,1110,11.2,59.6,0.7888


In [154]:
df.tail() # Extract the las 5 rows of the dataset(to check the data time period)

,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
9352,4/4/2005,10:00:00,3.1,1314,-200,13.5,1101,472,539,190,1374,1729,21.9,29.3,0.7568
9353,4/4/2005,11:00:00,2.4,1163,-200,11.4,1027,353,604,179,1264,1269,24.3,23.7,0.7119
9354,4/4/2005,12:00:00,2.4,1142,-200,12.4,1063,293,603,175,1241,1092,26.9,18.3,0.6406
9355,4/4/2005,13:00:00,2.1,1003,-200,9.5,961,235,702,156,1041,770,28.3,13.5,0.5139
9356,4/4/2005,14:00:00,2.2,1071,-200,11.9,1047,265,654,168,1129,816,28.5,13.1,0.5028


Data Description: Hourly time series data collected over approximately one year, 3/10/2004 - 4/4/2005
- Since the data was collected in Italy, seasonal climate variations across the four seasons should be considered.
- All variables are numeric
- A full year consists of 8,760 hours, so this dataset slightly exceeds one year, allowing for stable observation of annual seasonal patterns.

In [155]:
df.info() # Check the data types and counts of variables in the dataset

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9357 entries, 0 to 9356
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Date           9357 non-null   object 
 1   Time           9357 non-null   object 
 2   CO(GT)         9357 non-null   float64
 3   PT08.S1(CO)    9357 non-null   int64  
 4   NMHC(GT)       9357 non-null   int64  
 5   C6H6(GT)       9357 non-null   float64
 6   PT08.S2(NMHC)  9357 non-null   int64  
 7   NOx(GT)        9357 non-null   int64  
 8   PT08.S3(NOx)   9357 non-null   int64  
 9   NO2(GT)        9357 non-null   int64  
 10  PT08.S4(NO2)   9357 non-null   int64  
 11  PT08.S5(O3)    9357 non-null   int64  
 12  T              9357 non-null   float64
 13  RH             9357 non-null   float64
 14  AH             9357 non-null   float64
dtypes: float64(5), int64(8), object(2)
memory usage: 1.1+ MB


In [156]:
df.shape # Print the shape (dimensions) of the DataFrame

(9357, 15)

In [157]:
df.describe() # Summary data statistics

,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
count,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000
mean,-34.207524,1048.990061,-159.090093,1.865683,894.595276,168.616971,794.990168,58.148873,1391.479641,975.072032,9.778305,39.485380,-6.837604
std,77.657170,329.832710,139.789093,41.380206,342.333252,257.433866,321.993552,126.940455,467.210125,456.938184,43.203623,51.216145,38.976670
min,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000
25%,0.600000,921.000000,-200.000000,4.000000,711.000000,50.000000,637.000000,53.000000,1185.000000,700.000000,10.900000,34.100000,0.692300
50%,1.500000,1053.000000,-200.000000,7.900000,895.000000,141.000000,794.000000,96.000000,1446.000000,942.000000,17.200000,48.600000,0.976800
75%,2.600000,1221.000000,-200.000000,13.600000,1105.000000,284.000000,960.000000,133.000000,1662.000000,1255.000000,24.100000,61.900000,1.296200
max,11.900000,2040.000000,1189.000000,63.700000,2214.000000,1479.000000,2683.000000,340.000000,2775.000000,2523.000000,44.600000,88.700000,2.231000


### Data Cleaning
**Remove missing value**: Exclude observation where C6H6(GT) or CO(GT) equal -200.

In [158]:
filter_df = df[(df["C6H6(GT)"] != -200) & (df["CO(GT)"] != -200)] #!= means "not equal to," so rows with that value are excluded (removed)
filter_df # Print the cleaned dataset

,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
0,3/10/2004,18:00:00,2.6,1360,150,11.9,1046,166,1056,113,1692,1268,13.6,48.9,0.7578
1,3/10/2004,19:00:00,2.0,1292,112,9.4,955,103,1174,92,1559,972,13.3,47.7,0.7255
2,3/10/2004,20:00:00,2.2,1402,88,9.0,939,131,1140,114,1555,1074,11.9,54.0,0.7502
3,3/10/2004,21:00:00,2.2,1376,80,9.2,948,172,1092,122,1584,1203,11.0,60.0,0.7867
4,3/10/2004,22:00:00,1.6,1272,51,6.5,836,131,1205,116,1490,1110,11.2,59.6,0.7888
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9352,4/4/2005,10:00:00,3.1,1314,-200,13.5,1101,472,539,190,1374,1729,21.9,29.3,0.7568
9353,4/4/2005,11:00:00,2.4,1163,-200,11.4,1027,353,604,179,1264,1269,24.3,23.7,0.7119
9354,4/4/2005,12:00:00,2.4,1142,-200,12.4,1063,293,603,175,1241,1092,26.9,18.3,0.6406
9355,4/4/2005,13:00:00,2.1,1003,-200,9.5,961,235,702,156,1041,770,28.3,13.5,0.5139


In [159]:
filter_df.shape # Cehck the changes after data cleaning (verify how many observations were removed).

(7344, 15)

In [160]:
filter_df.describe() # Summary statistics after removing all rows where CO(GT) or C6H6(GT) equals -200.

,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
count,7344.000000,7344.000000,7344.000000,7344.000000,7344.000000,7344.000000,7344.000000,7344.000000,7344.000000,7344.000000,7344.000000,7344.000000,7344.000000
mean,2.129711,1110.580746,-150.224401,10.275735,947.198121,226.081563,826.920071,96.650327,1444.752723,1043.512936,17.770425,49.060076,0.989453
std,1.436472,218.681331,153.788325,7.441068,265.471608,227.144019,256.648430,85.089998,350.344170,405.569613,8.862688,17.451563,0.399889
min,0.100000,647.000000,-200.000000,0.200000,387.000000,-200.000000,322.000000,-200.000000,551.000000,221.000000,-1.900000,9.200000,0.184700
25%,1.100000,946.000000,-200.000000,4.600000,743.000000,90.000000,649.000000,73.000000,1203.000000,744.750000,11.200000,35.400000,0.698100
50%,1.800000,1075.000000,-200.000000,8.500000,919.000000,176.000000,795.000000,107.000000,1447.000000,990.000000,16.900000,49.300000,0.959700
75%,2.800000,1246.000000,-200.000000,14.300000,1125.250000,320.000000,960.000000,139.000000,1673.000000,1305.000000,23.800000,62.500000,1.258600
max,11.900000,2040.000000,1189.000000,63.700000,2214.000000,1479.000000,2683.000000,333.000000,2775.000000,2523.000000,44.600000,88.700000,2.180600


# Prediction of C6H6(GT)

## Loss Function
Suppose there is some prediction (call it c). The squared error loss for a data point (call it y1) and a prediction (c) is

\\[L(y_1, c) = (y_1 - c)^2\\]

For a given set of data ($y_1, ..., y_n$ - our C6H6(GT) values), we could consider the mean sqaured error:

*Root Mean Sqaure Error* = $RMSE(c)$ = $\sqrt{\frac{1}{n}\sum_{1=1}^{n}L(y_i, c)}$ = $\sqrt{\frac{1}{n}\sum_{i=1}^{n}(y_i-c)^2}$

As a prediction, we now want to choose the value of $c$ that minimizes this equation.



## Grid Search Algorithm
implementing a grid search to find the optimal value of $c$ based off.



### (1) Just $y$
Pseudo cod for using just the $y'$s and no other variables.

1. Create a *grid* of values for $c$. Look at the first and third quartiles for the `C6H6(GT)` variable to consider reasonable values for $c$.

In [161]:
### Find Q1 and Q3 of C6H6(GT) ###
y = filter_df["C6H6(GT)"]

q1 = y.quantile(0.25)
q3 = y.quantile(0.75)

print("Q1:", q1)
print("Q3:", q3)

Q1: 4.6
Q3: 14.3


In [162]:
## Divide the interval between q1 and q3 into a fine grid to generate candidate values for c ##
import numpy as np
grid = np.linspace(q1, q3, 1000)

print("c 후보값:", grid[:5]) #Extract 5 candidate values

c 후보값: [4.6        4.60970971 4.61941942 4.62912913 4.63883884]


2. Create a squared error loss function that takes in $y$ and $c$ that outputs $(y − c)^2$.

In [163]:
#For each candidate c, compute the RMSE when used as the prediction##

In [164]:
def squared_error(y, c): ## Define the squared error loss function
    return (y-c) ** 2

3. Create q a root mean squared error (objective) function that takes in $y$ and $c$ that outputs $\sqrt{\frac{1}{n}\sum_{1=1}^{n}L(y_i, c)}$ (You can put step 2 and 3 into one function if you want.)

In [165]:
def rmse(y, c): #Define the RMSE function
    return np.sqrt(np.mean(squared_error(y, c)))

4. Use a list comprehensive to loop over the grid of $c$ values, finding the $RMSE$ for each value of $c$.

In [166]:
rmse_list = [rmse(y,c) for c in grid] # list comprehension: RMSE value corresponding to the i-th candidate value of c
print(rmse_list[:5]) #print the RMSE values for the 5 candidate c values

[np.float64(9.35820070736372), np.float64(9.352314968689592), np.float64(9.346435610689953), np.float64(9.340562645413627), np.float64(9.33469608492665)]


5. Determine which value of $c$ gives the optimal (smallest) $RMSE$.

In [167]:
opt_index = np.argmin(rmse_list) # argmin returns the index of the smallest value
opt_index

np.int64(585)

In [168]:
opt_c = grid[opt_index] #return the grid value at that index
opt_rmse = rmse_list[opt_index] #return the RMSE value at that index

print("optimal c:", opt_c)
print("Minimum RMSE:", opt_rmse)

optimal c: 10.280180180180182
Minimum RMSE: 7.440562418757215


6. Report that as the prediction!

In [169]:
print(f"Constnat prediction for C6H6(GT) is c = {opt_c:.4f}") #.4f formats the number to display up to 4 decimal places
print(f"Miminum RMSE over the grid = {opt_rmse:.4f}")

Constnat prediction for C6H6(GT) is c = 10.2802
Miminum RMSE over the grid = 7.4406


7. Wrap the above into a function that takes in a column of data and outputs the value of $c$.

In [170]:
def grid_search_constant(y_sd, num_point = 1000):
    y_sd = np.asarray(y_sd)
    q1_sd = np.quantile(y_sd, 0.25)
    q3_sd = np.quantile(y_sd, 0.75)

    grid_sd = np.linspace(q1_sd, q3_sd, 1000)

    rmse_list_sd = [np.sqrt(np.mean((y_sd-c_sd) ** 2)) for c_sd in grid_sd]

    opt_index_sd = np.argmin(rmse_list_sd)
    opt_c_sd = grid[opt_index_sd]
    opt_rmse_sd = rmse_list_sd[opt_index_sd]

    return opt_c_sd

In [171]:
opt_c_sd = grid_search_constant(filter_df["C6H6(GT)"])
print(opt_c_sd)

10.280180180180182


In [172]:
opt_c_pt08 = grid_search_constant(filter_df["PT08.S1(CO)"])
opt_c_pt08

np.float64(9.92092092092092)

### (2) Using $y$ and another numeric variable $x$:
Next, you’ll implement the grid search to find the optimal
pair of values for $b_0$ and $b_1$ using `PT08.S1(CO)` as your x variable and `C6H6(GT)` as your $y$ variable.

The pseudo code is very similar to that above, but your grid now has two-dimensions!

In [173]:
## define x, y (.values converts the columns into Numpy arrays)
x_G2 = filter_df["PT08.S1(CO)"].values
y_G2 = filter_df["C6H6(GT)"].values

* You’ll need to populate a grid of b0 and b1 values that you want to consider.

  – Use $b_0$ values from -25 to -15 with increments of 0.1.

  – Use $b_1$ values from -5 to 5 with increments of 0.01

In [174]:
b0_grid = np.arange(-25, -15 + 0.1, 0.1) # +0.1, +0.01 are added because np.arange() may not include the end value (start, and, step)
b1_grid = np.arange(-5, 5 + 0.01, 0.01)

print("(1) Number of b0 values: ", len(b0_grid)) # Number
print("b0 values: ", b0_grid)                    # Value
print("(2) Number of b1 values: ", len(b1_grid))
print("b1 values: ", b1_grid)

(1) Number of b0 values:  101
b0 values:  [-25.  -24.9 -24.8 -24.7 -24.6 -24.5 -24.4 -24.3 -24.2 -24.1 -24.  -23.9
 -23.8 -23.7 -23.6 -23.5 -23.4 -23.3 -23.2 -23.1 -23.  -22.9 -22.8 -22.7
 -22.6 -22.5 -22.4 -22.3 -22.2 -22.1 -22.  -21.9 -21.8 -21.7 -21.6 -21.5
 -21.4 -21.3 -21.2 -21.1 -21.  -20.9 -20.8 -20.7 -20.6 -20.5 -20.4 -20.3
 -20.2 -20.1 -20.  -19.9 -19.8 -19.7 -19.6 -19.5 -19.4 -19.3 -19.2 -19.1
 -19.  -18.9 -18.8 -18.7 -18.6 -18.5 -18.4 -18.3 -18.2 -18.1 -18.  -17.9
 -17.8 -17.7 -17.6 -17.5 -17.4 -17.3 -17.2 -17.1 -17.  -16.9 -16.8 -16.7
 -16.6 -16.5 -16.4 -16.3 -16.2 -16.1 -16.  -15.9 -15.8 -15.7 -15.6 -15.5
 -15.4 -15.3 -15.2 -15.1 -15. ]
(2) Number of b1 values:  1001
b1 values:  [-5.   -4.99 -4.98 ...  4.98  4.99  5.  ]


* Report your optimal b0 and b1 combination.

In [175]:
### define RMSE function ###
def rmse_GridSearch_2dim(x_G2, y_G2, b0, b1):
    y_hat = b0 + b1 * x_G2
    return np.sqrt(np.mean((y_G2 - y_hat) ** 2))

In [176]:
## find the optimal combination ##
opt_rmse = np.inf #Assume the minimum RMSE is infinity (a very large starting value for comparison)
opt_b0 = None     # None indicates that the value has not been assigned yet
opt_b1 = None

for b0 in b0_grid:
  for b1 in b1_grid:
      cur_rmse = rmse_GridSearch_2dim(x_G2, y_G2, b0, b1)
      if cur_rmse < opt_rmse:
         opt_rmse = cur_rmse
         opt_b0 = b0
         opt_b1 = b1

print("Optimal b0:", opt_b0)
print("Optimal b1:", opt_b1)
print("Minimum RMSE:", opt_rmse)

Optimal b0: -22.99999999999997
Optimal b1: 0.02999999999989278
Minimum RMSE: 3.5429053908306782


* Create a function that takes in an x and and y column and outputs the optimal values.

In [177]:
def grid_search_2dim(x, y):
  x = np.asarray(x)
  y = np.asarray(y)

  b0_grid = np.arange(-25, -15 + 0.1, 0.1)
  b1_grid = np.arange(-5, 5 + 0.01, 0.01)

  opt_rmse = np.inf
  opt_b0 = None
  opt_b1 = None

  for b0 in b0_grid:
      for b1 in b1_grid:
          y_hat = b0 + b1 * x
          cur_rmse = np.sqrt(np.mean((y - y_hat) ** 2))

          if cur_rmse < opt_rmse:
              opt_rmse = cur_rmse
              opt_b0 = b0
              opt_b1 = b1

  return opt_b0, opt_b1

In [178]:
opt_b1, opt_b1 = grid_search_2dim(x_G2, y_G2)

print("Optimal b0:", opt_b0)
print("Optimal b1:", opt_b1)

Optimal b0: -22.99999999999997
Optimal b1: 0.02999999999989278


* Then use these values to predict a new C6H6(GT) for a PT08.S1(CO) of 946, 1075, and 1246.

In [179]:
new_x = np.array([946, 1075, 1246], dtype = float)
predicted_y = opt_b0 + opt_b1 * new_x

In [180]:
for xi, yi in zip(new_x, predicted_y):
    print(f"Predicted C6H6(GT) for PT08.S1(CO = {xi}: {yi})")

Predicted C6H6(GT) for PT08.S1(CO = 946.0: 5.3799999998985975)
Predicted C6H6(GT) for PT08.S1(CO = 1075.0: 9.249999999884764)
Predicted C6H6(GT) for PT08.S1(CO = 1246.0: 14.379999999866428)


## Gradient Descent Algorithm

### (1) Just y
First the case where we only consider the $y′$s and no $x′$s.

**Basic idea:**
* Start with an initial guess (use a value somewhat away from $\bar{y}$)
* Update the guess based on an approximation to the derivative (slope) of the RMSE and a “step-size” (generally some small value) `new_value = old_value - (slope) * step_size`
* Check if the movement (`|new_value - old_value|`) exceeds some small numeric tolerance. If so, we are still moving quite a bit and should repeat the steps above. If not, we’ve possibly settled very near a minimum and can stop. (Which type of loop is idea for this type of situation?) We should also set
a maximum number of iterations so we don’t repeat forever!

In [181]:
y_D1 = filter_df["C6H6(GT)"].values
np.mean(y_D1)

np.float64(10.275735294117647)

In [182]:
cur_c = 0.0 #initial assumption C6H6(GT) is 0

Gradient Descent Pseudo-code:
1. Create a squared error loss function that takes in y and c and outputs $(y − c)^2$.

In [183]:
def squared_error_G2(y_D1, c_D1):
    return (y_D1 - c_D1) ** 2

In [184]:
print("Test:",squared_error_G2(5, 3))

Test: 4


2. Create a root mean squared error (objective) q function that takes in y and c and outputs

RMSE(c) = $\sqrt{\frac{1}{n}\sum_{i=1}^{n}(y_i-c)^2}$


In [185]:
def rmse_D1(y_D1, c_D1):
    return np.sqrt(np.mean(squared_error_G2(y_D1, c_D1)))

In [186]:
print("Test:", rmse_D1(np.array([5, 6, 7]), 6))

Test: 0.816496580927726


3. Create a difference quotient function to approximate the slope of the tangent line that takes in the guess $c$, a small change to $c$ - I’ll call it $Δ$, and $y$ that outputs
\\[diff_{quotient} = \frac{RMSE(c + Δ) − RMSE(c)}{Δ}\\]


In [187]:
def diff_quotient(y_D1, c_D1, delta=0.001):
    return (rmse_D1(y_D1, c_D1 + delta) - rmse_D1(y_D1, c_D1)) / delta

4. Pick a starting value for `cur_c`.

In [188]:
delta = 0.001
slope_at0 = diff_quotient(y_D1, cur_c, delta) # Print the slope at the initial value: large magnitude -> steep curve
print("Slope at c = 0:", slope_at0)

Slope at c = 0: -0.809946859430255


5. Evaluate the difference quotient at `cur_c`.

In [189]:
slope_atmean = diff_quotient(y_D1, np.mean(y_D1), delta) # Print the slope at the mean value: small magnitude -> close to the minimum
print("Slope at c = mean:", slope_atmean)

Slope at c = mean: 6.719923195674937e-05


6. Update the value by moving (a small step) in the negative direction of the difference quotient

`new_c = cur_c - diff_quotient * step_size`

In [190]:
step_size = 0.01

slope = diff_quotient(y_D1, cur_c, delta)
new_c = cur_c - slope * step_size
movement = abs(new_c - cur_c)

print("cur_c:", cur_c)
print("slope:", slope)
print("new_c:", new_c)
print("movement:", movement)

cur_c: 0.0
slope: -0.809946859430255
new_c: 0.00809946859430255
movement: 0.00809946859430255


7. Check if `abs(new_c - cur_c) < num_tol` where `num_tol` is a small value.
* If so, update the `cur_c` to be the `new_c` value and stop.
* If not, update the value of `cur_c` to `new_c` and repeat steps 5-7.
  
  – Put in a safety that stops the loop after a maximum number of iterations is reached (even if `abs(new_c - cur_c) < num_tol` is not met)

In [191]:
num_tol = 0.0001

for i_D1 in range(2000):
    slope = diff_quotient(y_D1, cur_c, delta)
    new_c = cur_c - slope*step_size
    if abs(new_c - cur_c) < num_tol:
       cur_c = new_c
       print("Converged at iteration:", i_D1+1)
       break

    cur_c = new_c

print("Final cur_c:", cur_c)


Final cur_c: 9.235704036103156


8. Use the last value as the prediction!

In [192]:
pred_c = cur_c
print("Prediction c:", pred_c)

Prediction c: 9.235704036103156


9. Wrap the above into a function!

In [193]:
def gradient_descent_constant(y_D1, start_c = 0.0, delta = 0.001, step_size = 0.01, num_tol = 0.0001, max_iter = 20000):
    y_D1 = np.asarray(y_D1, dtype = float)
    cur_c = float(start_c)

    def diff_quotient(y_D1, c_D1, delta=0.001):
        return (rmse_D1(y_D1, c_D1 + delta) - rmse_D1(y_D1, c_D1)) / delta

    for i in range(max_iter):
        slope = diff_quotient(y_D1, cur_c, delta)
        new_c = cur_c - slope * step_size

        if abs(new_c - cur_c) < num_tol:
            cur_c = new_c
            return cur_c, i+1

        cur_c = new_c

    return cur_c, max_iter

In [194]:
opt_c_D1, n_iter = gradient_descent_constant(y_D1, start_c = 0.0, delta = 0.001, step_size = 0.01, num_tol = 0.0001, max_iter = 4000)

print("Optimal c:", opt_c_D1)
print("Iteration:", n_iter)

Optimal c: 10.200993520073336
Iteration: 3966


* Run your algorithm on the `C6H6(GT)` variable and determine the **optimal** prediction. Use 0 as your starting point.
  
  – Note: My $Δ$ was 0.001, my step size was 0.01, and my tolerance was 0.0001. It took around 4000 iterations for my algorithm to converge.


In [195]:
y_c6h6_D1 = filter_df["C6H6(GT)"].values

opt_c1, iter1 = gradient_descent_constant(y_c6h6_D1, start_c = 0.0, delta = 0.001, step_size = 0.01, num_tol = 0.0001, max_iter = 20000)

print("###### C6H6(GT) Result ######")
print("Optimal c:", opt_c1)
print("Iteration:", iter1)
print("mean:", np.mean(y_c6h6_D1))
print("Difference from mean:", abs(opt_c1 - np.mean(y_c6h6_D1)))

###### C6H6(GT) Result ######
Optimal c: 10.200993520073336
Iteration: 3966
mean: 10.275735294117647
Difference from mean: 0.07474177404431082


  
* Just to make sure your algorithm generalizes, run it using the `PT08.S1(CO)` variable as the response (this shouldn’t involve any new functions, just new function calls!). Use 1100 as your starting point.

  – Note: My $Δ$ was 0.001, my step size was 0.1, and my tolerance was 0.0001. It took around 8500 iterations for my algorithm to converge.

In [196]:
y_pt08_D1 = filter_df["PT08.S1(CO)"].values

opt_c2, iter2 = gradient_descent_constant(y_pt08_D1, start_c = 1100, delta = 0.001, step_size = 0.1, num_tol = 0.0001, max_iter = 40000)

print("###### PT08.S1(CO) Result ######")
print("Optimal c:", opt_c2)
print("Iteration:", iter2)
print("mean:", np.mean(y_pt08_D1))
print("Difference from mean:", abs(opt_c2 - np.mean(y_pt08_D1)))

###### PT08.S1(CO) Result ######
Optimal c: 1110.3616956683077
Iteration: 8483
mean: 1110.5807461873637
Difference from mean: 0.21905051905605433


### (2) y and anotehr numeric variable

In [197]:
x_D2 = filter_df["PT08.S1(CO)"].values
y_D2 = filter_df["C6H6(GT)"].values

1. Create a squared error loss function that takes in $y$, $x$, $b0$, and $b1$ that outputs $(y − b_0 − b_1x)^2$

In [198]:
def squared_error_D2(y_D2, x_D2, b0_D2, b1_D2):
    return (y_D2 - b0_D2 - b1_D2 * x_D2) ** 2

In [199]:
y_test = np.array([10, 12])
x_test = np.array([100, 200])
b0_test, b1_test = -20, 0.1

squared_error_D2(y_test, x_test, b0_test, b1_test)

array([400., 144.])

2. Create a mean squared error (objective) function that takes in the same values that outputs

RMSE($b_0$, $b_1$) = $\sqrt{\frac{1}{n}\sum_{i=1}^{n}(y - b_0 - b_1 x)^2}$


In [200]:
def rmse_obj_D2(y_D2, x_D2, b0_D2, b1_D2):
    return np.sqrt(np.mean(squared_error_D2(y_D2, x_D2, b0_D2, b1_D2)))

In [201]:
rmse_obj_D2(y_test, x_test, b0_test, b1_test)

np.float64(16.492422502470642)

3. Create a difference quotient function for $b_0$ to approximate the slope of the tangent line in the direction of $b_0$. It should take in $y$, $x$, $b_0$, a small change to $b_0$ - I’ll call it $Δ_0$, and $b_1$ that outputs

\\[diff_{quotient_{b0}} = \frac{RMSE(b_0 + Δ_0, b_1) − RMSE(b_0, b_1)}{Δ_0}\\]

In [202]:
def diff_quotient_b0(y_D2, x_D2, b0_D2, b1_D2, delta0 = 0.005):
    return (rmse_obj_D2(y_D2, x_D2, b0_D2 + delta0, b1_D2) - rmse_obj_D2(y_D2, x_D2, b0_D2, b1_D2)) / delta0

4. Create a difference quotient function for $b_1$ to approximate the slope of the tangent line in the direction of $b_1$. It should take in $y$, $x$, $b_0$, $b_1$ and a small change to $b_1$ - I’ll call it $Δ_1$ that outputs

\\[diff_{quotient_{b1}} = \frac{RMSE(b_0,b_1 + Δ_1) − RMSE(b_0, b_1)}{Δ_1}\\]



In [203]:
def diff_quotient_b1(y_D2, x_D2, b0_D2, b1_D2, delta1 = 0.005):
    return (rmse_obj_D2(y_D2, x_D2, b0_D2, b1_D2 + delta1) - rmse_obj_D2(y_D2, x_D2, b0_D2, b1_D2)) / delta1

5. Pick starting values for $b_0$ and $b_1$ (say `cur_b0` and `cur_b1`).

In [204]:
cur_b0 = -20.0
cur_b1 = 0.0

6. Evaluate the difference quotient for $b_0$ at the `cur_b0` and `cur_b1` values.


In [205]:
delta0 = 0.005
dq_b0 = diff_quotient_b0(y_D2, x_D2, cur_b0, cur_b1, delta0 = delta0)
dq_b0

np.float64(-0.9710990375836559)

7. Update the value of $b_0$ by moving (a small step) in the negative direction of the difference quotient

`new_b0 = cur_b0 - diff_quotient_b0 * step_size_b0`


In [206]:
step_size_b0 = 0.5
new_b0 = cur_b0 - dq_b0 * step_size_b0

new_b0

np.float64(-19.514450481208172)

8. Evaluate the difference quotient for $b_1$ at the `new_b0` and `cur_b1` values.

In [208]:
delta1 = 0.005
dq_b1 = diff_quotient_b1(y_D2, x_D2, new_b0, cur_b1, delta1 = delta1)
dq_b1

np.float64(-1122.3179362419842)

9. Update the value of $b_1$ by moving (a small step) in the negative direction of the difference quotient

`new_b1 = cur_b1 - diff_quotient_b1 * step_size_b1`

In [209]:
step_size_b1 = 0.00005
new_b1 = cur_b1 - dq_b1 * step_size_b1
new_b1

np.float64(0.05611589681209921)

10. Check if the distance between the `cur_b0`, `cur_b1` vector to the `new_b0`, `new_b1` vector is less than some small tolerance. (Hint: Look up the euclidean distance between two vectors, there is a function in numpy to calculate it - or just code it up yourself).
* If so, update the `cur_b0` and `cur_b1` to be the `new_b0` and `new_b1` values and stop.
* If not, update the `cur_b0` and `cur_b1` to be the `new_b0` and `new_b1` values and repeat steps 6-10.

  – Put in a safety that stops the loop after a maximum number of iterations is reached (even if the tolerance is not met)

distance = $\sqrt{(new_{b0} - cur_{b0})^2 + (new_{b1} - cur_{b1})^2}$


In [210]:
tol = 0.0001

dist = np.linalg.norm(np.array([new_b0, new_b1]) - np.array([cur_b0, cur_b1]))
dist

np.float64(0.4887814737426142)

In [213]:
if dist < tol:
    cur_b0, cur_b1 = new_b0, new_b1 # converge
    print("Converged:",cur_b0, cur_b1)
else:
    cur_b0, cur_b1 = new_b0, new_b1 # no converge
    print("Not Converged, Continue updated:", cur_b0, cur_b1,)

Not Converged, Continue updated: -19.514450481208172 0.05611589681209921


11. Use the last values as the estimates for $b_0$ and $b_1$!

In [219]:
###6-10 for ###
max_iter_D2 = 100000

for i_D2 in range(max_iter_D2):
    dq_b0 = diff_quotient_b0(y_D2, x_D2, cur_b0, cur_b1, delta0 = delta0)
    new_b0 = cur_b0 - dq_b0 * step_size_b0
    dq_b1 = diff_quotient_b1(y_D2, x_D2, new_b0, cur_b1, delta1 = delta1)
    new_b1 = cur_b1 - dq_b1 * step_size_b1
    dist = np.linalg.norm(np.array(new_b0, new_b1) - np.array([cur_b0, cur_b1]))

    if dist < tol:
        cur_b0, cur_b1 = new_b0, new_b1
        print(f"Converged at iteration {i_D2+1}")
        break
    cur_b0, cur_b1 = new_b0, new_b1

print("####Gradient Descent (b0, b1) Result######")
print("b0:", cur_b0)
print("b1:", cur_b1)
print("iterations:", i_D2 + 1)
print("RMSE:", rmse_obj_D2(y_D2, x_D2, cur_b0, cur_b1))

####Gradient Descent (b0, b1) Result######
b0: -21.817105022717413
b1: 0.05474852642948789
iterations: 100000
RMSE: 29.432524304542124


Use initial starting values of -20 for the intercept and 0 for the slope. Report your optimal b0 and b1 combination. Then use these values to predict a new `C6H6(GT)` for a `PT08.S1(CO)` of 946, 1075, and 1246.

* Note: I used a step size of 0.5 for the intercept, 0.00005 for the slope, a delta of 0.005 for both the intercept and slope, and a tolerance of 0.0001.

  – I used 100,000 iterations and my algorithm took 5-10 minutes to run. I hit my limit and got reasonably close to the truth but not super close (-23.18 and 0.0017, respectively).
  
  – That’s close enough here! We’d need to do some fancier adaptive step size to make this work better.

In [222]:
cur_b0_i = -20.0
cur_b1_i = 0.0

delta0_i = 0.005
delta1_i = 0.005
step_size_b0_i = 0.5
step_size_b1_i = 0.00005
tol_i = 0.0001
max_iter_i = 100000

for i_it in range(max_iter_i):
    dq_b0_i = diff_quotient_b0(y_D2, x_D2, cur_b0_i, cur_b1_i, delta0 = delta0_i)
    new_b0_i = cur_b0_i - dq_b0_i * step_size_b0_i
    dq_b1_i = diff_quotient_b1(y_D2, x_D2, new_b0_i, cur_b1_i, delta1 = delta1_i)
    new_b1_i = cur_b1_i - dq_b1_i * step_size_b1_i
    dist_i = np.linalg.norm(np.array([new_b0_i, new_b1_i]) - np.array([cur_b0_i, cur_b1_i]))

    if dist_i < tol_i:
        cur_b0_i, cur_b1_i = new_b0_i, new_b1_i
        print(f"Converged at iteration {i_it+1}")
        break
    cur_b0_i, cur_b1_i = new_b0_i, new_b1_i

print("####Gradient Descent (b0, b1) Result######")
print("b0:", cur_b0_i)
print("b1:", cur_b1_i)
print("iterations:", i_it + 1)
print("RMSE:", rmse_obj_D2(y_D2, x_D2, cur_b0_i, cur_b1_i))

####Gradient Descent (b0, b1) Result######
b0: -22.30482975822852
b1: -0.0014903666811793767
iterations: 100000
RMSE: 35.097272964373865


**Issue: The estimated b1 value does not come out positive**

In [223]:
new_x_i = np.array([946, 1075, 1246], dtype = float)
pred_y_i = cur_b0_i + cur_b1_i * new_x_i

for x_i, y_i in zip(new_x_i, pred_y_i):
    print(f"PT08.S1(CO)={x_i}: Predicted C6H6(GT)={y_i}")

PT08.S1(CO)=946.0: Predicted C6H6(GT)=-23.71471663862421
PT08.S1(CO)=1075.0: Predicted C6H6(GT)=-23.90697394049635
PT08.S1(CO)=1246.0: Predicted C6H6(GT)=-24.161826642978024
